In [1]:
import os

In [75]:
%pwd

'C:\\Users\\hp\\Documents\\Projects\\Python\\SkinCancer\\SkinCancer-Classification'

In [74]:
import os
os.chdir("C:/Users/hp/Documents/Projects/Python/SkinCancer/SkinCancer-Classification")


In [14]:
os.chdir("../")

In [15]:
%pwd

'C:\\Users\\hp\\Documents\\Projects\\Python\\SkinCancer'

In [76]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL1: str
    source_URL2: str
    local_data_file1: Path
    local_data_file2: Path
    unzip_dir: Path

In [77]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [84]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL1=config.source_URL1,
            source_URL2=config.source_URL2,
            local_data_file1=config.local_data_file1,
            local_data_file2=config.local_data_file2,
            unzip_dir=config.unzip_dir)
        
        return data_ingestion_config

In [85]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [88]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
        
    def download_file(self):
            
        if not os.path.exists(self.config.local_data_file1):
            filename1, headers1 = request.urlretrieve(
                url=self.config.source_URL1, 
                filename=self.config.local_data_file1)
            logger.info(f"{filename1} downloaded! with following info: \n{headers1}")
        else:
            logger.info(f"File already exists: {self.config.local_data_file1} of size: {get_size(Path(self.config.local_data_file1))}")

        # Descargar archivo de la segunda URL
        if not os.path.exists(self.config.local_data_file2):
            filename2, headers2 = request.urlretrieve(
                url=self.config.source_URL2, 
                filename=self.config.local_data_file2)
            logger.info(f"{filename2} downloaded! with following info: \n{headers2}")
        else:
            logger.info(f"File already exists: {self.config.local_data_file2} of size: {get_size(Path(self.config.local_data_file2))}")
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extract the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        if os.path.exists(self.config.local_data_file1):
            
            with zipfile.ZipFile(self.config.local_data_file1, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"Extracted {self.config.local_data_file1} to {unzip_path}")

    # Extraer el segundo archivo ZIP
        if os.path.exists(self.config.local_data_file2):
            with zipfile.ZipFile(self.config.local_data_file2, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"Extracted {self.config.local_data_file2} to {unzip_path}")

In [89]:
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-14 12:14:43,875: INFO: common: Reading the yaml file from config\config.yaml]
[2024-10-14 12:14:43,891: INFO: common: Reading the yaml file from params.yaml]
[2024-10-14 12:14:43,891: INFO: common: Directory created at artifacts]
[2024-10-14 12:14:43,891: INFO: common: Directory created at artifacts/data_ingestion]
[2024-10-14 12:14:43,906: INFO: 1937193348: File already exists: artifacts/data_ingestion/data.zip of size: ~25490.92 kb]
[2024-10-14 12:14:43,906: INFO: 1937193348: File already exists: artifacts/data_ingestion/data2.zip of size: ~25153.93 kb]
[2024-10-14 12:14:47,728: INFO: 1937193348: Extracted artifacts/data_ingestion/data.zip to artifacts/data_ingestion]
[2024-10-14 12:14:51,009: INFO: 1937193348: Extracted artifacts/data_ingestion/data2.zip to artifacts/data_ingestion]
